In [14]:
import sklearn
from sklearn import datasets
import pandas as pd

In [62]:
from sklearn.model_selection import cross_val_score
def load_data(data, input_features, target_feature, cat_features=None, num_features=None, txt_features=None):
    all_features = input_features + [target_feature]
    print(data.values.shape)
    data = data[all_features]
    data.dropna(subset=[target_feature], inplace=True)

    # change categorical features to numeric code
    if(cat_features!=None):
        data[cat_features] = data[cat_features].astype('category')
        data[cat_features] = data[cat_features].apply(lambda x: x.cat.codes)
    # replace nan with 0 in numerical features
    if(num_features!=None):
        data[num_features] = data[num_features].fillna(0.)
        for feature in num_features:
            data[feature] = data[feature].apply(lambda x: replace_string(x))
    if(txt_features!=None):
        if txt_features:
            data[txt_features] = data[txt_features].fillna('')

    return data


In [46]:
data=sklearn.datasets.fetch_20newsgroups(data_home=None, subset='train', categories=None, shuffle=True, random_state=42, remove=(), download_if_missing=True)

In [27]:
data = pd.DataFrame(list(zip(data['data'],data['target'])))

In [29]:
data.columns = ['text','category']

In [42]:
data=load_data(data,['text'],'category',txt_features=['text'])

(11314, 2)


In [47]:
from sklearn.feature_extraction.text import CountVectorizer
#Count vectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(data.data)
print(X_train_counts.shape)
#counts
count_vect.vocabulary_

(11314, 130107)

In [57]:
#frequencies
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(11314, 130107)

In [64]:
model.get_depth()

197

In [145]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=0,criterion ='gini',max_depth=50)

In [141]:
prediction=model.fit(X_train_tf,data.target)

In [70]:
prediction.score(X_train_tf,data.target)#(self, X, y, sample_weight=None)

0.9999116139296447

In [146]:
cross_val_score(model, X_train_tf, data.target, cv=3)

array([0.55523179, 0.56241717, 0.55231014])

In [93]:
def loguniform(low=0, high=1, size=None):
    return stats.reciprocal(np.exp(low), np.exp(high))

In [94]:

import numpy as np

from time import time
import scipy.stats as stats
#from scipy.stats import loguniform


from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.linear_model import SGDClassifier

# get some data
X, y = load_digits(return_X_y=True)

# build a classifier
clf = SGDClassifier(loss='hinge', penalty='elasticnet',
                    fit_intercept=True)


# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {'average': [True, False],
              'l1_ratio': stats.uniform(0, 1),
              'alpha': loguniform(1e-4, 1e0)}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)

start = time()
random_search.fit(X, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

/Users/Rgao/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


RandomizedSearchCV took 4.13 seconds for 20 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.732 (std: 0.028)
Parameters: {'alpha': 2.6417350951260086, 'average': False, 'l1_ratio': 0.14839141435592695}

Model with rank: 2
Mean validation score: 0.698 (std: 0.051)
Parameters: {'alpha': 1.331553941750644, 'average': False, 'l1_ratio': 0.315674117038799}

Model with rank: 3
Mean validation score: 0.594 (std: 0.044)
Parameters: {'alpha': 1.224576524194162, 'average': False, 'l1_ratio': 0.4500802010879611}



/Users/Rgao/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [163]:
 param_dist = {'splitter':['best','random'],
                  'max_features':["auto","sqrt","log2",None],
                  'class_weight': ['balanced',None],
                  'min_samples_leaf':[1,5,10,25,50,100],
                  'min_samples_split':[2,5,10,25,50,100],
                  'max_depth':[10,25,50,100,200]
              }
n_iter_search = 20
random_search = RandomizedSearchCV(model, param_distributions=param_dist,
                                   n_iter=n_iter_search)

In [164]:
start = time()
random_search.fit(X_train_tf, data.target)

/Users/Rgao/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=DecisionTreeClassifier(class_weight=None,
                                                    criterion='gini',
                                                    max_depth=50,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort=False,
                                                    random_state=0,
                                                    splitter='best')

In [165]:
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)


RandomizedSearchCV took 37.32 seconds for 20 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.466 (std: 0.012)
Parameters: {'splitter': 'best', 'min_samples_split': 10, 'min_samples_leaf': 100, 'max_features': None, 'max_depth': 50, 'class_weight': None}

Model with rank: 2
Mean validation score: 0.416 (std: 0.017)
Parameters: {'splitter': 'random', 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': None, 'max_depth': 25, 'class_weight': None}

Model with rank: 3
Mean validation score: 0.365 (std: 0.022)
Parameters: {'splitter': 'best', 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 100, 'class_weight': None}



In [ ]:
def randomizedsearch(X,y,model=DecisionTreeClassifier(random_state=0,criterion ='gini'),
                     n_iter_search=20,
                      param_dist = {'splitter':['best','random'],
                                    'max_features':["auto","sqrt","log2",None],
                                    'class_weight': ['balanced',None],
                                    'min_samples_leaf':[1,5,10,25,50,100],
                                    'min_samples_split':[2,5,10,25,50,100],
                                    'max_depth':[10,25,50,100,200]}):
    random_search = RandomizedSearchCV(model, param_distributions=param_dist,
                                   n_iter=n_iter_search)
    start = time()
    random_search.fit(X_train_tf, data.target)
    print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
    report(random_search.cv_results_)
    return(randomized_search)
    
def gridsearch(X,y,model=DecisionTreeClassifier(random_state=0,criterion ='gini'),
               param_grid = {'splitter':['best','random'],
                                    'max_features':["auto","sqrt","log2",None],
                                    'class_weight': ['balanced',None],
                                    'min_samples_leaf':[1,5,10,25,50,100],
                                    'min_samples_split':[2,5,10,25,50,100],
                                    'max_depth':[10,25,50,100,200]}):
    grid_search = GridSearchCV(model, param_grid=param_grid)
    start = time()
    grid_search.fit(X, y)
    print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
    report(grid_search.cv_results_)
    return(grid_search)

In [169]:
randomizedsearch(X_train_tf,data.target)

/Users/Rgao/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


RandomizedSearchCV took 99.75 seconds for 20 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.607 (std: 0.002)
Parameters: {'splitter': 'best', 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': None, 'max_depth': 200, 'class_weight': None}

Model with rank: 2
Mean validation score: 0.540 (std: 0.012)
Parameters: {'splitter': 'random', 'min_samples_split': 25, 'min_samples_leaf': 10, 'max_features': None, 'max_depth': 50, 'class_weight': None}

Model with rank: 3
Mean validation score: 0.487 (std: 0.016)
Parameters: {'splitter': 'random', 'min_samples_split': 25, 'min_samples_leaf': 50, 'max_features': None, 'max_depth': 50, 'class_weight': 'balanced'}



In [ ]:
gridsearch(X_train_tf,data.target)

/Users/Rgao/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [103]:
model.get_params().keys()

dict_keys(['class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'presort', 'random_state', 'splitter'])